<a href="https://colab.research.google.com/github/LFIORITO1/labo1/blob/main/Code.Phyton.class%2011.18.2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [65]:
import pandas as pd

from sklearn.model_selection import train_test_split, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import cohen_kappa_score


In [66]:
data = pd.read_csv("/content/drive/MyDrive/Petfinder/train.csv.zip", index_col="PetID").select_dtypes(exclude="O").fillna(0)
X_train, X_test, y_train, y_test = train_test_split(data.drop("AdoptionSpeed", axis=1), data.AdoptionSpeed, test_size=0.1, random_state=42)

kf = KFold(5)
folds = [(X_train.index[train_idx], X_train.index[valid_idx]) for train_idx, valid_idx in kf.split(X_train)]


In [67]:
metrica = lambda y_true, y_probs: cohen_kappa_score(y_true, y_probs.values.argmax(axis=1), weights= 'quadratic')

In [84]:
Estimators = [100, 200, 300]
#results = ...
df = pd.DataFrame(columns=list(["Valid_result","test_result"])) # 2 porque el index es la primera columna. la columna 0



for n_estimators in Estimators:
  valid_probs = []
  test_probs = pd.DataFrame(index=X_test.index, columns=list(range(5))).fillna(0)
  for train_petId, valid_petId in folds:
    Xt = X_train.loc[train_petId]
    yt = y_train.loc[train_petId]

    model = RandomForestClassifier(n_estimators=n_estimators)
    model.fit(Xt, yt)

    Xv = X_train.loc[valid_petId]
    valid_probs.append(
        pd.DataFrame(model.predict_proba(Xv), index=valid_petId)
    )

    test_probs = test_probs + pd.DataFrame(model.predict_proba(X_test), index=X_test.index)


  valid_probs = pd.concat(valid_probs)
  test_probs = test_probs / len(folds)
  valid_result = metrica(y_train, valid_probs.loc[y_train.index])
  test_result = metrica(y_test, test_probs.loc[y_test.index])

  df.loc[n_estimators, "Valid_result" ] = valid_result
  df.loc[n_estimators, "test_result" ] = test_result



In [85]:
print(df)

    Valid_result test_result
100     0.321094    0.350756
200     0.329763    0.363658
300     0.325833    0.353811


In [46]:
test_result, valid_result

NameError: ignored

In [7]:
pd.DataFrame(index = X_test.index, columns=list(range(5))).fillna(0) # data fram de 0s

,0,1,2,3,4
PetID,,,,,
3f8824a3b,0,0,0,0,0
9238eb7fc,0,0,0,0,0
f0a1f2b90,0,0,0,0,0
7d028bdea,0,0,0,0,0
8377bfe97,0,0,0,0,0
...,...,...,...,...,...
0d1067e63,0,0,0,0,0
c93e86f5d,0,0,0,0,0
8bc346340,0,0,0,0,0


In [15]:
test_probs.shape, X_test.shape

((1500, 5), (1500, 22))